# Fork and Merge Hunt Flows

No one knows what is the correct threat hypothesis from the beginning. This makes threat hunting hard, but also makes it exciting---developing the threat hypothesis along the hunting procedure. You may find some part of the hunt correct or useful but the following part is not. You may want to fork hunt flows to verify hypothesis variants or even different threat hypotheses. You may want to merge hunt flows and apply the same following hunt steps onward. Hunt flows to Kestrel is like control-flow to a classic detection/investigation program. In addition, the hunt flow is dynamically composed, updated, and developed to fit the best threat hypothesis.

## What you will learn

0. What is composable hunt flow?
1. How to fork a hunt flow?
2. How to merge two hunt flows?
3. Exercise: identify hunt flow branching

### 0. What is composable hunt flow?

Reuse a hunt flow, cut part of a hunt flow and concatenate it with another, fork a hunt flow, merge hunt flows. These are all operations to compose hunt flow from small hunt flow segments. Visit the Kestrel documentation to learn more about [composable hunt flow](https://kestrel.readthedocs.io/en/latest/language.html#composable-hunt-flow).

### 1. How to fork a hunt flow?

Use a variable in multiple upcoming hunt steps to fork a hunt flow into multple branches:

In [1]:
# the trunk hunt flow
conns = GET network-traffic
        FROM file:///tmp/lab101.json
        WHERE [network-traffic:dst_port > 0]
        START t'2021-04-03T00:00:00Z' STOP t'2021-04-04T00:00:00Z' 

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,network-traffic*,process*,user-account*,x-ecs-destination*,x-ecs-network*,x-ecs-process*,x-ecs-source*,x-ecs-user*,x-oca-asset*,x-oca-event*
conns,network-traffic,504,504,504,504,1594,926,504,0,504,504,504,504,504,504,504,504,504


In [2]:
# fork a branch to get external IPv4
conns_ext = GET network-traffic
            FROM conns
            WHERE [network-traffic:dst_ref.type = 'ipv4-addr' AND network-traffic:dst_ref.value NOT LIKE '10.%']

ipv4_ext = FIND ipv4-addr ACCEPTED conns_ext

DISP ipv4_ext ATTR value

value 
 
 
 
 
 104.97.85.29 
 
 
 104.97.85.50 
 
 
 13.86.101.172 
 
 
 23.199.63.11 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 conns_ext 
 network-traffic 
 10 
 10 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 ipv4_ext 
 ipv4-addr 
 4 
 10 
 504 
 504 
 1590 
 926 
 504 
 504 
 504 
 504 
 504 
 504 
 504 
 504 
 504 
 504 
 504 
 
 
 *Number of related records cached.

In [3]:
# fork another branch to find processes associated with the network-traffic
procs = FIND process CREATED conns

proc_parents = FIND process CREATED procs

DISP proc_parents ATTR name

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,network-traffic*,process*,user-account*,x-ecs-destination*,x-ecs-network*,x-ecs-process*,x-ecs-source*,x-ecs-user*,x-oca-asset*,x-oca-event*
procs,process,5,505,506,506,1596,927,505,504,501,505,504,504,505,504,505,505,505
proc_parents,process,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# fork a small branch from the second branch to grab the child processes
proc_children = FIND process CREATED BY procs

DISP proc_children ATTR name

name 
 
 
 
 
 rdpclip.exe 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 proc_children 
 process 
 1 
 1 
 506 
 506 
 1596 
 927 
 505 
 504 
 505 
 505 
 504 
 504 
 505 
 504 
 505 
 505 
 505 
 
 
 *Number of related records cached.

### 2. How to merge two hunt flows?

Just do `varX + varY` to merge variables, a.k.a., the results of hunt flows:

In [5]:
proc_all = procs + proc_children

DISP proc_all ATTR name, pid

name 
 pid 
 
 
 
 
 rdpclip.exe 
 13624 
 
 
 svchost.exe 
 1068 
 
 
 svchost.exe 
 1552 
 
 
 svchost.exe 
 1632 
 
 
 svchost.exe 
 1776 
 
 
 svchost.exe 
 3612 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 proc_all 
 process 
 6 
 506 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

### 3. Exercise: identify hunt flow branching

Identify branches of hunt flows in **4. Group Entities in a Variable.ipynb**